#### This notebook demonstrates the use of the Reject Option Classification (ROC) post-processing algorithm for bias mitigation.
- The debiasing function used is implemented in the `RejectOptionClassification` class.
- Divide the dataset into training, validation, and testing partitions.
- Train classifier on original training data.
- Estimate the optimal classification threshold, that maximizes balanced accuracy without fairness constraints.
- Estimate the optimal classification threshold, and the critical region boundary (ROC margin) using a validation set for the desired constraint on fairness. The best parameters are those that maximize the classification threshold while satisfying the fairness constraints.
- The constraints can be used on the following fairness measures:
    * Statistical parity difference on the predictions of the classifier
    * Average odds difference for the classifier
    * Equal opportunity difference for the classifier
- Determine the prediction scores for testing data. Using the estimated optimal classification threshold, compute accuracy and fairness metrics.
- Using the determined optimal classification threshold and the ROC margin, adjust the predictions. Report accuracy and fairness metric on the new predictions.

In [15]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
from tqdm import tqdm
from warnings import warn
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pickle

## Huangrui's Dataset

In [16]:
default_mappings = {
    'label_maps': [{1.0: 'Did recid.', 0.0: 'No recid.'}],
    'protected_attribute_maps': [{0.0: 'Male', 1.0: 'Female'},
                                 {1.0: 'Caucasian', 0.0: 'Not Caucasian'}]
}
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
class CompasDataset(StandardDataset):
    """ProPublica COMPAS Dataset.

    See :file:`aif360/data/raw/compas/README.md`.
    """

    def __init__(self, label_name='Y', favorable_classes=[1],
                 protected_attribute_names=['sex'],
                 privileged_classes=[[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_keep=[],
                 features_to_drop=[], na_values=[],
                 custom_preprocessing=None,
                 metadata=default_mappings,
                 path='./Huangrui/recidivism/recidivism_test1.csv'):

    
        df = pd.read_csv(path,index_col=False)
        df.rename(columns={'Probationerssex_Female': 'sex'}, inplace=True)
        df.drop(["Probationerssex_Male","Probationerssex_Notascertained"], axis=1, inplace=True)
        numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
        code_continuous(df,numericals, 5)
        
        super(CompasDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)


#### Load dataset and specify options

In [17]:
## import dataset
dataset_used = "compas" 

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
        
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
np.random.seed(1)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

In [18]:
experiments_info = {}
budget = 1
for K in range(1, 6):
    dataset_orig_train= CompasDataset(path="./Huangrui/recidivism/recidivism_train{}.csv".format(K),protected_attribute_names=['sex'],
                privileged_classes=[[1]])
    dataset_orig_test= CompasDataset(path="./Huangrui/recidivism/recidivism_test{}.csv".format(K),protected_attribute_names=['sex'],
                privileged_classes=[[1]])
    #only use the budget% of the training data
    # Get the dataset and split into train and test
    if budget == 0.01:
        #for budget = 0.01 use seed 
        dataset_orig_train,_ = dataset_orig_train.split([budget ], shuffle=True,seed = 43)
    else:
        dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)

    # Lasso linear classifier and predictions
    X_train = dataset_orig_train.features
    y_train = dataset_orig_train.labels.ravel()
    lmod = pickle.load(open('experiments/recidivism'+str(K)+'_sex_bmodel.pkl','rb'))["clf"]
   
    y_train_pred = lmod.predict(X_train)

    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_train_pred.labels = y_train_pred.reshape(-1,1)
    sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
    dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)

    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    X_test = dataset_orig_test_pred.features
    y_test = dataset_orig_test_pred.labels
    dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)

    #### Best threshold for classification only (no fairness)
    num_thresh = 100
    ba_arr = np.zeros(num_thresh)
    class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
    for idx, class_thresh in enumerate(class_thresh_arr):
        
        fav_inds = dataset_orig_train_pred.scores > class_thresh
        dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
        dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label
        
        classified_metric_orig_train = ClassificationMetric(dataset_orig_train,
                                                dataset_orig_train_pred, 
                                                unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        
        ba_arr[idx] = 0.5*(classified_metric_orig_train.true_positive_rate()\
                        +classified_metric_orig_train.true_negative_rate())

    best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
    best_class_thresh = class_thresh_arr[best_ind]

    # print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
    # print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)
    #### Estimate optimal parameters for the ROC method
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    low_class_thresh=0.01, high_class_thresh=0.99,
                                    num_class_thresh=100, num_ROC_margin=50,
                                    metric_name=metric_name,
                                    metric_ub=metric_ub, metric_lb=metric_lb)
    ROC = ROC.fit(dataset_orig_train, dataset_orig_train_pred)
    # print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
    # print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

    # Metrics for the train set
    fav_inds = dataset_orig_train_pred.scores > best_class_thresh
    dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
    dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label

    # display(Markdown("#### train set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_train_bef = compute_metrics(dataset_orig_train, dataset_orig_train_pred, 
                    unprivileged_groups, privileged_groups)

    # Transform the validation set
    dataset_transf_train_pred = ROC.predict(dataset_orig_train_pred)

    # display(Markdown("#### train set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_train_aft = compute_metrics(dataset_orig_train, dataset_transf_train_pred, 
                    unprivileged_groups, privileged_groups)
    # Testing: Check if the metric optimized has not become worse
    assert np.abs(metric_train_aft[metric_name]) <= np.abs(metric_train_bef[metric_name])

    # Metrics for the test set
    fav_inds = dataset_orig_test_pred.scores > best_class_thresh
    dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
    dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                    unprivileged_groups, privileged_groups)
    # Metrics for the transformed test set
    dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                    unprivileged_groups, privileged_groups)

    #自己计算error, 不是balanced accuracy！！！
    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels), "Equal opportunity difference": metric_test_aft["Equal opportunity difference"]}

C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

Balanced accuracy = 0.7769
Statistical parity difference = 0.1606
Disparate impact = 1.5405
Average odds difference = 0.0939
Equal opportunity difference = 0.1089
Theil index = 0.1320
Balanced accuracy = 0.6370
Statistical parity difference = 0.1534
Disparate impact = 1.2276
Average odds difference = 0.0967
Equal opportunity difference = 0.0469
Theil index = 0.0689
Balanced accuracy = 0.7239
Statistical parity difference = 0.1605
Disparate impact = 1.7213
Average odds difference = 0.1063
Equal opportunity difference = 0.1413
Theil index = 0.1910
Balanced accuracy = 0.6192
Statistical parity difference = 0.1161
Disparate impact = 1.1740
Average odds difference = 0.0918
Equal opportunity difference = 0.1194
Theil index = 0.0937
K = 1, budget = 1
The Error for the test dataset is 0.4468
The Equal opportunity difference for the test dataset is 0.1194


C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

Balanced accuracy = 0.7705
Statistical parity difference = 0.1606
Disparate impact = 1.5782
Average odds difference = 0.1065
Equal opportunity difference = 0.1519
Theil index = 0.1420
Balanced accuracy = 0.5922
Statistical parity difference = 0.0938
Disparate impact = 1.1181
Average odds difference = 0.0554
Equal opportunity difference = 0.0241
Theil index = 0.0613
Balanced accuracy = 0.7406
Statistical parity difference = 0.2041
Disparate impact = 1.9542
Average odds difference = 0.1390
Equal opportunity difference = 0.1443
Theil index = 0.1711
Balanced accuracy = 0.5590
Statistical parity difference = 0.1066
Disparate impact = 1.1332
Average odds difference = 0.0788
Equal opportunity difference = 0.0511
Theil index = 0.0700
K = 2, budget = 1
The Error for the test dataset is 0.5316
The Equal opportunity difference for the test dataset is 0.05112


C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

Balanced accuracy = 0.7757
Statistical parity difference = 0.1750
Disparate impact = 1.6325
Average odds difference = 0.0996
Equal opportunity difference = 0.0964
Theil index = 0.1355
Balanced accuracy = 0.5858
Statistical parity difference = 0.1384
Disparate impact = 1.1813
Average odds difference = 0.0921
Equal opportunity difference = 0.0357
Theil index = 0.0616
Balanced accuracy = 0.6798
Statistical parity difference = 0.1363
Disparate impact = 1.2698
Average odds difference = 0.0832
Equal opportunity difference = 0.0847
Theil index = 0.1203
Balanced accuracy = 0.5361
Statistical parity difference = 0.0591
Disparate impact = 1.0665
Average odds difference = 0.0468
Equal opportunity difference = 0.0427
Theil index = 0.0588
K = 3, budget = 1
The Error for the test dataset is 0.5646
The Equal opportunity difference for the test dataset is 0.04275


C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

Balanced accuracy = 0.7691
Statistical parity difference = 0.1710
Disparate impact = 1.6066
Average odds difference = 0.1079
Equal opportunity difference = 0.1318
Theil index = 0.1391
Balanced accuracy = 0.6273
Statistical parity difference = 0.1500
Disparate impact = 1.2184
Average odds difference = 0.0931
Equal opportunity difference = 0.0401
Theil index = 0.0696
Balanced accuracy = 0.7142
Statistical parity difference = 0.1262
Disparate impact = 1.3962
Average odds difference = 0.0913
Equal opportunity difference = 0.1494
Theil index = 0.1743
Balanced accuracy = 0.6042
Statistical parity difference = 0.0853
Disparate impact = 1.1116
Average odds difference = 0.0599
Equal opportunity difference = 0.0683
Theil index = 0.0714
K = 4, budget = 1
The Error for the test dataset is 0.477
The Equal opportunity difference for the test dataset is 0.06834


C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_13944\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

Balanced accuracy = 0.7672
Statistical parity difference = 0.2001
Disparate impact = 1.7603
Average odds difference = 0.1399
Equal opportunity difference = 0.1806
Theil index = 0.1383
Balanced accuracy = 0.5438
Statistical parity difference = 0.1085
Disparate impact = 1.1282
Average odds difference = 0.0744
Equal opportunity difference = 0.0249
Theil index = 0.0559
Balanced accuracy = 0.7578
Statistical parity difference = 0.1335
Disparate impact = 1.4806
Average odds difference = 0.0881
Equal opportunity difference = 0.0949
Theil index = 0.1571
Balanced accuracy = 0.5443
Statistical parity difference = 0.0730
Disparate impact = 1.0831
Average odds difference = 0.0465
Equal opportunity difference = -0.0037
Theil index = 0.0545
K = 5, budget = 1
The Error for the test dataset is 0.5576
The Equal opportunity difference for the test dataset is -0.003686


In [19]:
experiments_info

{'K = 1, budget = 1': {'Error': 0.4467811158798283,
  'Equal opportunity difference': 0.119421906693712},
 'K = 2, budget = 1': {'Error': 0.5315586088449978,
  'Equal opportunity difference': 0.051115241635687814},
 'K = 3, budget = 1': {'Error': 0.5646200085873766,
  'Equal opportunity difference': 0.042746327776860826},
 'K = 4, budget = 1': {'Error': 0.47702876771146413,
  'Equal opportunity difference': 0.06833676168086977},
 'K = 5, budget = 1': {'Error': 0.5575601374570447,
  'Equal opportunity difference': -0.0036855036855036882}}